# Simulation of a single chain

In [1]:
import unittest
import math
import pandas as pd

script="""
MonteCarlo
Configuration cubic_box_length 40 periodic0 false periodic1 false periodic2 false \
    cutoff 20 particle_type0 /feasst/plugin/chain/forcefield/chain5.fstprt
Potential Model LennardJones VisitModel VisitModelIntra cutoff 1
ThermoParams beta 1 chemical_potential0 1
Metropolis
TrialAdd particle_type 0
Run until_num_particles 1
RemoveTrial name TrialAdd
TrialPivot weight 1 tunable_param 20
TrialCrankshaft weight 1 tunable_param 20
TrialGrowFile file_name grow_chain5.txt
set_variable trials_per 1e4
Log trials_per trials_per file_name chain.txt
Movie trials_per trials_per file_name chain.xyz
Energy trials_per_write trials_per file_name chain_en.txt
CheckEnergy trials_per trials_per tolerance 1e-10
Tune
Run num_trials 1e3
RadiusOfGyration file_name rg.txt
Run num_trials 1e5
"""

with open('grow_chain5.txt', 'w') as file: file.write(
'TrialGrowFile\n\n\
particle_type 0 site 0 regrow true\n\
bond true mobile_site 1 anchor_site 0\n\
bond true mobile_site 2 anchor_site 1\n\
bond true mobile_site 3 anchor_site 2\n\
bond true mobile_site 4 anchor_site 3\n')
    
with open('script.txt', 'w') as file: file.write(script)
import subprocess
syscode = subprocess.call("../../../build/bin/fst < script.txt > script.log", shell=True, executable='/bin/bash')
with open('script.log', 'r') as file: print(file.read(), '\n', 'exit:', syscode)

# FEASST version: v0.19.0-85-gdff8cb12e2-dirty-hwh/newtutorials
Configuration cubic_box_length 40 cutoff 20 particle_type0 /home/hwh/feasst/plugin/chain/forcefield/chain5.fstprt periodic0 false periodic1 false periodic2 false  
Potential Model LennardJones VisitModel VisitModelIntra cutoff 1  
ThermoParams beta 1 chemical_potential0 1  
Metropolis  
TrialAdd particle_type 0  
Run until_num_particles 1  
# initializing random number generator with seed: 1652899566
RemoveTrial name TrialAdd  
TrialPivot tunable_param 20 weight 1  
TrialCrankshaft tunable_param 20 weight 1  
TrialGrowFile file_name grow_chain5.txt  
Log file_name chain.txt trials_per 1e4  
Movie file_name chain.xyz trials_per 1e4  
Energy file_name chain_en.txt trials_per_write 1e4  
CheckEnergy tolerance 1e-10 trials_per 1e4  
Tune  
Run num_trials 1e3  
RadiusOfGyration file_name rg.txt  
Run num_trials 1e5  
 
 exit: 0


In [3]:
class TestChain5RG(unittest.TestCase):
    def test_rg_en(self):
        rg=pd.read_csv('rg.txt')
        self.assertAlmostEqual(0.8806, rg['average'][0], delta=3*rg['block_stdev'][0])
        en=pd.read_csv('chain_en.txt')
        self.assertAlmostEqual(-2.050, en['average'][0], delta=3*en['block_stdev'][0])
unittest.main(argv=[''], verbosity=2, exit=False)

test_rg_en (__main__.TestChain5RG) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!